In [19]:
import cv2
import mediapipe as mp
import numpy as np
import math

In [30]:
def MiddleFingerDetector(landmarks):
    middle_finger_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value]
    middle_finger_pip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value]
    middle_finger_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP.value]
    middle_diff = math.pow(middle_finger_tip[1] - middle_finger_mcp[1], 2) + math.pow(middle_finger_tip[0] - middle_finger_mcp[0], 2)
    middle_diff /= 2

    index_finger_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP.value]
    index_finger_pip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_PIP.value]
    index_finger_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP.value]
    index_diff = math.pow(index_finger_tip[1] - index_finger_mcp[1], 2) + math.pow(index_finger_tip[0] - index_finger_mcp[0], 2)


    ring_finger_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP.value]
    ring_finger_pip = landmarks[mp_hands.HandLandmark.RING_FINGER_PIP.value]
    ring_finger_mcp = landmarks[mp_hands.HandLandmark.RING_FINGER_MCP.value]
    ring_diff = math.pow(ring_finger_tip[1] - ring_finger_mcp[1], 2) + math.pow(ring_finger_tip[0] - ring_finger_mcp[0], 2)

    pinky_finger_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP.value]
    pinky_finger_pip = landmarks[mp_hands.HandLandmark.PINKY_PIP.value]
    pinky_finger_mcp = landmarks[mp_hands.HandLandmark.PINKY_MCP.value]
    pinky_diff = math.pow(pinky_finger_tip[1] - pinky_finger_mcp[1], 2) + math.pow(pinky_finger_tip[0] - pinky_finger_mcp[0], 2)

    middle_finger_is_raised = False
    if middle_diff > index_diff and middle_diff > ring_diff and middle_diff > pinky_diff:
        middle_finger_is_raised = True

    return middle_finger_is_raised


In [38]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

def MiddleFingerDetector(landmarks):
    middle_finger_tip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_TIP.value]
    middle_finger_pip = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_PIP.value]
    middle_finger_mcp = landmarks[mp_hands.HandLandmark.MIDDLE_FINGER_MCP.value]
    middle_diff = math.pow(middle_finger_tip[1] - middle_finger_mcp[1], 2) + math.pow(middle_finger_tip[0] - middle_finger_mcp[0], 2)
    middle_diff /= 2

    index_finger_tip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_TIP.value]
    index_finger_pip = landmarks[mp_hands.HandLandmark.INDEX_FINGER_PIP.value]
    index_finger_mcp = landmarks[mp_hands.HandLandmark.INDEX_FINGER_MCP.value]
    index_diff = math.pow(index_finger_tip[1] - index_finger_mcp[1], 2) + math.pow(index_finger_tip[0] - index_finger_mcp[0], 2)


    ring_finger_tip = landmarks[mp_hands.HandLandmark.RING_FINGER_TIP.value]
    ring_finger_pip = landmarks[mp_hands.HandLandmark.RING_FINGER_PIP.value]
    ring_finger_mcp = landmarks[mp_hands.HandLandmark.RING_FINGER_MCP.value]
    ring_diff = math.pow(ring_finger_tip[1] - ring_finger_mcp[1], 2) + math.pow(ring_finger_tip[0] - ring_finger_mcp[0], 2)

    pinky_finger_tip = landmarks[mp_hands.HandLandmark.PINKY_TIP.value]
    pinky_finger_pip = landmarks[mp_hands.HandLandmark.PINKY_PIP.value]
    pinky_finger_mcp = landmarks[mp_hands.HandLandmark.PINKY_MCP.value]
    pinky_diff = math.pow(pinky_finger_tip[1] - pinky_finger_mcp[1], 2) + math.pow(pinky_finger_tip[0] - pinky_finger_mcp[0], 2)

    middle_finger_is_raised = False
    if middle_diff > index_diff and middle_diff > ring_diff and middle_diff > pinky_diff:
        middle_finger_is_raised = True

    return middle_finger_is_raised


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks_np = np.array([(lm.x, lm.y) for lm in hand_landmarks.landmark], dtype=np.float32)

            x, y = np.min(landmarks_np, axis=0)
            w, h = np.max(landmarks_np, axis=0) - (x, y)

            scale_factor = 1.5
            x -= w * (scale_factor - 1) / 2
            y -= h * (scale_factor - 1) / 2
            w *= scale_factor
            h *= scale_factor

            detected = MiddleFingerDetector(landmarks_np)
            rect_color = (0, 0, 255) if detected else (0, 255, 0)

            cv2.rectangle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])),
                          (int((x + w) * frame.shape[1]), int((y + h) * frame.shape[0])), rect_color, 2)

            if detected:
                # Crop the region within the rectangle
                hand_roi = frame[int(y * frame.shape[0]):int((y + h) * frame.shape[0]),
                                int(x * frame.shape[1]):int((x + w) * frame.shape[1])]

                # Apply an even larger Gaussian blur to the cropped region for more intensity
                blurred_hand_roi = cv2.GaussianBlur(hand_roi, (75, 75), 0)

                # Replace the original region with the more blurred one
                frame[int(y * frame.shape[0]):int((y + h) * frame.shape[0]),
                      int(x * frame.shape[1]):int((x + w) * frame.shape[1])] = blurred_hand_roi
            
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow('Hand Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


I0000 00:00:1702117398.154543       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 86), renderer: Apple M1
